<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.63
    Uninstalling yfinance-0.2.63:
      Successfully uninstalled yfinance-0.2.63
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-06-16 12:22:02
-------------------
qualified stocks: 92
with latest results: 29
still star stocks: 17
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  1.50 L
Current:  1.33 C
-------------------
Today PnL: -29.56 K (-0.22%)
Current PnL: -16.66 L (-11.61%)
CY Booked + Current PnL: -10.29 L (-7.17%)
-------------------
Total profit:  3.00 L
Total loss:  -19.67 L
-------------------
Total Booked + Current PnL: 16.69 L (14.32%)
Total Booked PnL: 33.35 L (28.62%)
Curr Year Booked PnL: 6.37 L (4.81%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.05 C
Est FTT PnL: 72.78 L (54.91%)
Deployed:  1.17 C
Current:  1.33 C
CAGR/XIRR %: 8.97%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,LICI,827.06,983.0,-5.02,H-LC,88.04,97000.0,11813.0,4249.0,0.40,13.87,4.38,18.85,35.0,2.78,0.74,29.15,XY25,NTT,INSURANCE
61,RELIANCE,1291.83,1526.0,5.36,X-LC,33.70,156513.0,15704.0,9813.0,0.56,11.15,6.27,18.13,17.0,1.60,1.19,23.58,XY25,NTT,REFINERIES
3,ADANIPORTS,1103.69,1583.0,-0.76,M-LC,41.30,99372.0,21010.0,13018.0,-0.38,26.81,13.10,43.43,77.0,1.61,0.76,33.56,XY24,NTT,MISC
77,TATAELXSI,7332.28,7332.0,-15.14,X-MC,65.22,89180.0,-13472.0,13466.0,-0.35,-13.12,15.10,-0.00,57.0,-1.00,0.68,35.56,OX40N,NTT,IT
88,VIPIND,488.80,489.0,-957.82,H-SC,89.13,81705.0,-13611.0,13653.0,3.57,-14.28,16.71,0.04,153.0,-1.00,0.62,61.15,OX40N,NTT,MISC


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
89,VOLTAS,1278.28,1929.20,-29.06,H-MC,55.43,193455.0,1713.0,95934.0,0.98,0.89,49.59,50.92,99.0,0.02,1.48,7.15,XY25,ATH,AC
10,BAJAJHFL,122.26,152.00,-7.52,H-LC,28.26,198932.0,-1574.0,50350.0,-0.27,-0.79,25.31,24.33,78.0,-0.03,1.52,14.72,X40N,BTT,FINANCE
57,PGHH,13388.00,18106.42,-20.27,X-MC,10.87,201030.0,210.0,70562.0,-0.33,0.10,35.10,35.24,53.0,0.00,1.53,4.79,X40,ATH,FMCG
34,HINDALCO,651.95,761.55,-13.37,H-LC,31.52,103368.0,-944.0,18482.0,0.66,-0.90,17.88,16.81,26.0,-0.05,0.79,15.00,X5K,ATH,METALS
36,HINDZINC,514.80,744.74,37.28,H-LC,96.74,102500.0,-460.0,46453.0,-0.34,-0.45,45.32,44.67,25.0,-0.01,0.78,32.28,X5K,ATH,METALS


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
88,VIPIND,488.80,489.0,-957.82,H-SC,89.13,81705.0,-13611.0,13653.0,3.57,-14.28,16.71,0.04,153.0,-1.00,0.62,61.15,OX40N,NTT,MISC
68,SFL,1016.35,1287.0,294.58,H-SC,77.17,178162.0,-84056.0,153879.0,3.50,-32.06,86.37,26.63,145.0,-0.55,1.36,10.77,XY24,NTT,MISC
73,STARHEALTH,551.56,761.0,0.51,M-SC,43.48,164167.0,-36049.0,112077.0,1.93,-18.01,68.27,37.97,215.0,-0.32,1.25,31.81,XY24,NTT,INSURANCE
20,CIPLA,1495.00,1795.0,-15.96,H-LC,54.35,102530.0,2365.0,17738.0,1.67,2.36,17.30,20.07,29.0,0.13,0.78,10.51,X40N,BTT,PHARMA
54,MEDANTA,1064.65,1486.0,3.83,X-MC,42.39,232430.0,26953.0,54365.0,1.54,13.12,23.39,39.58,67.0,0.50,1.77,24.55,XY24,NTT,HEALTHCARE


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TATAMOTORS,761.77,1065.00,-55.10,M-LC,13.04,247929.0,-28594.0,138667.0,-4.08,-10.34,55.93,39.81,69.0,-0.21,1.89,18.82,XY24,NTT,AUTO
85,VAIBHAVGBL,350.21,521.00,58.13,X-SC,21.74,105332.0,-46659.0,120784.0,-3.48,-30.70,114.67,48.77,61.0,-0.39,0.80,20.99,XR,NTT,JEWELLERY
22,COFFEEDAY,59.14,80.00,-55.73,L-SC,97.83,66701.0,-46848.0,86898.0,-3.12,-41.26,130.28,35.27,260.0,-0.54,0.51,63.25,XR,NTT,HOTELS
76,TANLA,917.30,1963.11,-30.98,H-SC,94.57,168932.0,-70483.0,343439.0,-2.90,-29.44,203.30,114.01,128.0,-0.21,1.29,54.94,AR,ATH,IT
83,UJJIVANSFB,52.77,53.00,48.19,M-SC,82.61,128817.0,-13662.0,14286.0,-2.77,-9.59,11.09,0.44,242.0,-0.96,0.98,52.04,OX40N,NTT,BANKS


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,UJJIVANSFB,52.77,53.0,48.19,M-SC,82.61,128817.0,-13662.0,14286.0,-2.77,-9.59,11.09,0.44,242.0,-0.96,0.98,52.04,OX40N,NTT,BANKS
77,TATAELXSI,7332.28,7332.0,-15.14,X-MC,65.22,89180.0,-13472.0,13466.0,-0.35,-13.12,15.10,-0.00,57.0,-1.00,0.68,35.56,OX40N,NTT,IT
75,SYMPHONY,1306.42,1306.0,-30.32,M-SC,2.17,148371.0,-22770.0,22716.0,-0.49,-13.30,15.31,-0.03,190.0,-1.00,1.13,6.64,OX40N,NTT,DURABLES
88,VIPIND,488.80,489.0,-957.82,H-SC,89.13,81705.0,-13611.0,13653.0,3.57,-14.28,16.71,0.04,153.0,-1.00,0.62,61.15,OX40N,NTT,MISC
19,CERA,8421.60,8422.0,-12.65,X-SC,78.26,90252.0,-19229.0,19233.0,-0.79,-17.56,21.31,0.00,65.0,-1.00,0.69,36.60,OX40N,NTT,CERAMICS


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,INFY,1461.46,2275.00,-9.55,X-LC,60.87,289855.0,28254.0,117362.0,1.08,10.80,40.49,55.67,1.0,0.24,2.21,17.56,X40,BTT,IT
79,TCS,3794.03,4998.00,-16.16,X-LC,38.04,272735.0,-23199.0,117112.0,1.48,-7.84,42.94,31.73,2.0,-0.20,2.08,9.17,X40,BTT,IT
35,HINDUNILVR,2413.81,2723.00,-16.95,X-LC,27.17,260680.0,-9667.0,44290.0,0.37,-3.58,16.99,12.81,5.0,-0.22,1.99,7.89,XY25,NTT,FMCG
15,BRITANNIA,4983.33,6446.05,11.23,X-LC,46.20,254978.0,25745.0,41536.0,-0.48,11.23,16.29,29.35,7.0,0.62,1.95,21.15,XY25,ATH,FMCG
56,NESTLEIND,2268.60,2755.00,-11.96,X-LC,39.13,279373.0,13947.0,42968.0,0.46,5.25,15.38,21.44,10.0,0.32,2.13,11.78,XY25,NTT,FMCG


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ADANIPORTS,1103.69,1583.0,-0.76,M-LC,41.3,99372.0,21010.0,13018.0,-0.38,26.81,13.1,43.43,77.0,1.61,0.76,33.56,XY24,NTT,MISC


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
33,HEROMOTOCO,4311.81,6039.03,0.48,H-MC,46.20,153178.0,2265.0,58192.0,0.99,1.50,37.99,40.06,100.0,0.04,1.17,25.29,AR,ATH,AUTO
18,CAMS,3643.00,5250.99,-0.67,H-SC,58.70,111611.0,9607.0,35414.0,0.14,9.42,31.73,44.14,125.0,0.27,0.85,27.19,X40N,ATH,MISC
3,ADANIPORTS,1103.69,1583.00,-0.76,M-LC,41.30,99372.0,21010.0,13018.0,-0.38,26.81,13.10,43.43,77.0,1.61,0.76,33.56,XY24,NTT,MISC
84,UNIONBANK,123.87,163.00,-4.91,M-LC,90.22,166570.0,25730.0,18756.0,0.18,18.27,11.26,31.59,86.0,1.37,1.27,44.69,XY24,NTT,BANKS
91,WIPRO,243.46,420.00,-5.63,M-LC,56.52,161442.0,10497.0,98964.0,0.07,6.95,61.30,72.51,68.0,0.11,1.23,11.21,XR,NTT,IT


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
90,WHIRLPOOL,1167.49,2270.00,-34.03,M-SC,47.83,192232.0,20611.0,141464.0,-1.58,12.01,73.59,94.43,194.0,0.15,1.47,41.88,XR,NTT,DURABLES
91,WIPRO,243.46,420.00,-5.63,M-LC,56.52,161442.0,10497.0,98964.0,0.07,6.95,61.30,72.51,68.0,0.11,1.23,11.21,XR,NTT,IT
40,INDIAMART,2327.09,4850.92,-49.23,H-SC,83.70,132129.0,8793.0,124968.0,0.98,7.13,94.58,108.45,122.0,0.07,1.01,31.08,AR,ATH,MISC
12,BANKINDIA,116.91,190.00,-30.66,M-MC,73.91,185314.0,5506.0,106908.0,-0.58,3.06,57.69,62.52,170.0,0.05,1.41,33.36,XR,NTT,BANKS
33,HEROMOTOCO,4311.81,6039.03,0.48,H-MC,46.20,153178.0,2265.0,58192.0,0.99,1.50,37.99,40.06,100.0,0.04,1.17,25.29,AR,ATH,AUTO


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,59.14,80.00,-55.73,L-SC,97.83,66701.0,-46848.0,86898.0,-3.12,-41.26,130.28,35.27,260.0,-0.54,0.51,63.25,XR,NTT,HOTELS
53,MASFIN,326.60,399.50,-22.31,H-SC,81.52,88125.0,-9855.0,31725.0,1.15,-10.06,36.00,22.32,141.0,-0.31,0.67,27.81,XR,ATH,FINANCE
51,LICI,827.06,983.00,-5.02,H-LC,88.04,97000.0,11813.0,4249.0,0.40,13.87,4.38,18.85,35.0,2.78,0.74,29.15,XY25,NTT,INSURANCE
3,ADANIPORTS,1103.69,1583.00,-0.76,M-LC,41.30,99372.0,21010.0,13018.0,-0.38,26.81,13.10,43.43,77.0,1.61,0.76,33.56,XY24,NTT,MISC
36,HINDZINC,514.80,744.74,37.28,H-LC,96.74,102500.0,-460.0,46453.0,-0.34,-0.45,45.32,44.67,25.0,-0.01,0.78,32.28,X5K,ATH,METALS


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,75.41,137.00,7383.33,M-SC,100.00,82104.0,-11706.0,88327.0,-0.93,-12.48,107.58,81.67,238.0,-0.13,0.63,59.00,XR,NTT,CERAMICS
86,VALIANTORG,512.64,838.00,-379.48,H-SC,98.91,118690.0,-14596.0,99189.0,-2.35,-10.95,83.57,63.47,142.0,-0.15,0.91,98.95,XR,NTT,CHEMICALS
22,COFFEEDAY,59.14,80.00,-55.73,L-SC,97.83,66701.0,-46848.0,86898.0,-3.12,-41.26,130.28,35.27,260.0,-0.54,0.51,63.25,XR,NTT,HOTELS
36,HINDZINC,514.80,744.74,37.28,H-LC,96.74,102500.0,-460.0,46453.0,-0.34,-0.45,45.32,44.67,25.0,-0.01,0.78,32.28,X5K,ATH,METALS
47,JPPOWER,18.73,26.20,-34.84,L-SC,95.65,132376.0,-9466.0,66042.0,-1.08,-6.67,49.89,39.88,258.0,-0.14,1.01,34.15,XY24,NTT,POWER


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.56
1,25,42.86
2,50,71.78


In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    38.75
LC    37.51
MC    21.99
Name: CurrAlloc%, dtype: float64

In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.51
XY25     14.18
X40      12.58
XR       10.43
X40N      9.88
OX40N     8.37
AR        8.02
X5K       2.32
X200      1.49
SR        1.19
Name: CurrAlloc%, dtype: float64

In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    19.41
X-LC    15.51
M-SC    14.17
H-LC    12.57
H-MC     8.62
M-LC     8.32
X-MC     7.66
M-MC     5.71
X-SC     2.75
L-SC     2.42
L-LC     1.11
Name: CurrAlloc%, dtype: float64

In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.48,-3.12,37.61
IT,11.36,-11.61,68.30
BANKS,8.03,-8.27,52.45
FINANCE,7.83,-17.55,60.01
MISC,6.59,-25.25,74.11
PAINTS,5.59,-19.21,42.70
ELECTRICAL,5.12,-4.45,49.68
HEALTHCARE,4.76,-3.40,33.61
AUTO,4.56,-12.95,62.61


In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2802476.0,25
XR,1091214.0,13
AR,1017507.0,9
X40,670795.0,9
XY25,530410.0,10
X40N,483003.0,9
OX40N,331776.0,11
SR,190127.0,2
X5K,97250.0,3


In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1952558.0,19
M-SC,1472512.0,18
X-LC,629864.0,9
M-MC,594956.0,5
H-LC,556538.0,11
H-MC,496874.0,7
X-MC,410880.0,6
M-LC,353268.0,6
L-SC,279887.0,4


In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,913182.0,6
M-SC,XY24,588916.0,5
H-SC,AR,522680.0,3
M-MC,XY24,400121.0,3
M-SC,XR,348551.0,4
X-LC,X40,315548.0,3
M-LC,XY24,235364.0,4
H-MC,XY24,228039.0,2
H-SC,XR,208529.0,3


In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 67.0 seconds
